# Assignment Module 2: Product Classification

The goal of this assignment is to implement a neural network that classifies smartphone pictures of products found in grocery stores. The assignment will be divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.


## Preliminaries: the dataset

The dataset you will be using contains natural images of products taken with a smartphone camera in different grocery stores:

<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Granny-Smith.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Pink-Lady.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Lemon.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Banana.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Vine-Tomato.jpg" width="150">
</p>
<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Yellow-Onion.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Green-Bell-Pepper.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Arla-Standard-Milk.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Oatly-Natural-Oatghurt.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Alpro-Fresh-Soy-Milk.jpg" width="150">
</p>

The products belong to the following 43 classes:
```
0.  Apple
1.  Avocado
2.  Banana
3.  Kiwi
4.  Lemon
5.  Lime
6.  Mango
7.  Melon
8.  Nectarine
9.  Orange
10. Papaya
11. Passion-Fruit
12. Peach
13. Pear
14. Pineapple
15. Plum
16. Pomegranate
17. Red-Grapefruit
18. Satsumas
19. Juice
20. Milk
21. Oatghurt
22. Oat-Milk
23. Sour-Cream
24. Sour-Milk
25. Soyghurt
26. Soy-Milk
27. Yoghurt
28. Asparagus
29. Aubergine
30. Cabbage
31. Carrots
32. Cucumber
33. Garlic
34. Ginger
35. Leek
36. Mushroom
37. Onion
38. Pepper
39. Potato
40. Red-Beet
41. Tomato
42. Zucchini
```

The dataset is split into training (`train`), validation (`val`), and test (`test`) set.

The following code cells download the dataset and define a `torch.utils.data.Dataset` class to access it. This `Dataset` class will be the starting point of your assignment: use it in your own code and build everything else around it.

In [36]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset.git

fatal: destination path 'GroceryStoreDataset' already exists and is not an empty directory.


In [1]:
from pathlib import Path
from PIL import Image
from torch import Tensor
from torch.utils.data import Dataset
from typing import List, Tuple

In [2]:
class GroceryStoreDataset(Dataset):

    def __init__(self, split: str, transform=None) -> None:
        super().__init__()

        self.root = Path("GroceryStoreDataset/dataset")
        self.split = split
        self.paths, self.labels = self.read_file()

        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img = Image.open(self.root / self.paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def read_file(self) -> Tuple[List[str], List[int]]:
        paths = []
        labels = []

        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                # path, fine-grained class, coarse-grained class
                path, _, label = line.replace("\n", "").split(", ")
                paths.append(path), labels.append(int(label))

        return paths, labels

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it on `GroceryStoreDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split of **around 60%**. You are free to achieve that however you want, except for a few rules you must follow:

- You **cannot** simply instantiate an off-the-self PyTorch network. Instead, you must construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you **cannot** use e.g. `torchvision.models.alexnet`.

- Justify every *design choice* you make. Design choices include network architecture, training hyperparameters, and, possibly, dataset preprocessing steps. You can either (i) start from the simplest convolutional network you can think of and add complexity one step at a time, while showing how each step gets you closer to the target ~60%, or (ii) start from a model that is already able to achieve the desired accuracy and show how, by removing some of its components, its performance drops (i.e. an *ablation study*). You can *show* your results/improvements however you want: training plots, console-printed values or tables, or whatever else your heart desires: the clearer, the better.

Don't be too concerned with your network performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded **more** points than a poorly experimentally validated model with higher accuracy.

## Part 2: fine-tune an existing network

Your goal is to fine-tune a pretrained **ResNet-18** model on `GroceryStoreDataset`. Use the implementation provided by PyTorch, do not implement it yourselves! (i.e. exactly what you **could not** do in the first part of the assignment). Specifically, you must use the PyTorch ResNet-18 model pretrained on ImageNet-1K (V1). Divide your fine-tuning into two parts:

1. First, fine-tune the Resnet-18 with the same training hyperparameters you used for your best model in the first part of the assignment.
1. Then, tweak the training hyperparameters in order to increase the accuracy on the validation split of `GroceryStoreDataset`. Justify your choices by analyzing the training plots and/or citing sources that guided you in your decisions (papers, blog posts, YouTube videos, or whatever else you find enlightening). You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split **between 80 and 90%**.

---
---

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import numpy as np
import random
from typing import List, Tuple
from torch import Tensor
from pathlib import Path
from torchvision import transforms as T, datasets
from PIL import Image
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

def fix_random(seed: int) -> None:
    """
        Fix all the possible sources of randomness.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Device: cpu


## Data preprocessing

In [14]:
import torchvision.transforms as T
from torch.utils.data import ConcatDataset

# Define the data augmentation transformations
NUM_CLASSES = 43
INPUT_WIDTH = 348
INPUT_HEIGHT = 348
BATCH_SIZE = 64
USE_DATA_AUGMENTATION = True
AUGMENTATION_FACTOR = 2

tsfms = T.Compose([
    T.ToTensor(),
    T.CenterCrop((INPUT_HEIGHT, INPUT_WIDTH)) # Some images are not the same size
])
augmentation_transforms = [T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.RandomResizedCrop((INPUT_HEIGHT, INPUT_WIDTH), scale=(0.8, 1.0)),
    T.ToTensor(),
    T.CenterCrop((INPUT_HEIGHT, INPUT_WIDTH)) # Some images are not the same size
]), T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.RandomResizedCrop((INPUT_HEIGHT, INPUT_WIDTH), scale=(0.8, 1.0)),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    T.ToTensor(),
    T.CenterCrop((INPUT_HEIGHT, INPUT_WIDTH)) # Some images are not the same size
])]

# Check if thge file 'train_dset.pt' exists
if Path('train_dset.pt').exists():
    train_dset = torch.load('train_dset.pt')
else:
    # Concatenate the original and augmented training datasets
    train_dset = GroceryStoreDataset(split="train", transform=tsfms)
    if USE_DATA_AUGMENTATION:
        print(f'Train dataset size before augmentation: {len(train_dset)}')
        for i in range(AUGMENTATION_FACTOR):
            for j in range(len(augmentation_transforms)):
                augmented_train_dset = GroceryStoreDataset(split="train", transform=augmentation_transforms[j])
                train_dset = ConcatDataset([train_dset, augmented_train_dset])
        print(f'Train dataset size after augmentation: {len(train_dset)}')
        # Dump the whole dataset for later use in order to avoid recomputing the augmentations
        torch.save(train_dset, 'train_dset.pt')


test_dset = GroceryStoreDataset(split="test", transform=tsfms)
val_dset = GroceryStoreDataset(split="val", transform=tsfms)

val_dl = DataLoader(val_dset, 128)
test_dl = DataLoader(test_dset, 128)
train_dl = DataLoader(train_dset, BATCH_SIZE)

In [5]:
def evaluate(model, test_dl, device):
    corrects = 0
    total = 0
    model = model.to(device)

    with torch.no_grad():
        model.eval()
        for image, labels in test_dl:
            image, labels = image.to(device), labels.to(device)
            pred_logits = model(image)
            _, preds = torch.max(pred_logits, 1)
            corrects += (preds == labels).sum().item()
            total += labels.shape[0]

    return corrects / total

## Part 1

### Basic Model

Each Convolutional block in the network has the following structure:
1. 2D Convolution
2. SiLU Activation Function
3. Max Pooling
4. Batch Normalization
5. Dropout

In [6]:
class ConvBlock(nn.Module):
    def __init__(self, kernel_size, in_channels, out_channels, activation):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.block = nn.Sequential(
            nn.Conv2d(
                in_channels = in_channels, 
                out_channels = out_channels, 
                kernel_size = kernel_size, 
                stride = 2, 
                padding = "valid"
            ),
            nn.BatchNorm2d(out_channels),
            activation,
            nn.Conv2d(
                in_channels = out_channels, 
                out_channels = out_channels*2, 
                kernel_size = kernel_size, 
                stride = 1, 
                padding = "same"
            ),
            nn.BatchNorm2d(out_channels*2),
            activation,
            nn.Conv2d(
                in_channels = out_channels*2, 
                out_channels = out_channels, 
                kernel_size = kernel_size, 
                stride = 1, 
                padding = "same"
            ),
            nn.BatchNorm2d(out_channels),
        )
        
        self.activation = activation
        
        self.skip_block = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding="valid")
    
    def forward(self, x):
        # Residual connection
        residual = self.skip_block(x)
        x = self.block(x) + residual
        return self.activation(x)
        

### First Model

In [15]:
class FirstModel(nn.Module):
    def __init__(self, input_dim, n_classes):
        super().__init__()
        
        self.stem_block = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=3, padding="valid"),
            nn.SiLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(p=0.05),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding="valid"),
            nn.SiLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(p=0.05)
        )
                
        self.conv_block_1 = ConvBlock(kernel_size=3, in_channels=128, out_channels=256, activation=nn.SiLU())
        self.conv_block_2 = ConvBlock(kernel_size=3, in_channels=256, out_channels=256, activation=nn.SiLU())
        # self.conv_block_3 = ConvBlock(kernel_size=3, in_channels=256, out_channels=512, activation=nn.SiLU())
        
        self.linear_block = nn.Sequential(
            nn.Linear(256, 1024),
            nn.SiLU(),
            nn.Dropout(p=0.2),
            nn.Linear(1024, 2048),
            nn.SiLU(),
            nn.Dropout(p=0.2),
            #nn.Linear(2048, 2048),
            #nn.SiLU(),
            #nn.Dropout(p=0.2),
            nn.Linear(2048, n_classes)
        )        
        
    def forward(self, x):
        x = self.stem_block(x)
        
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        # x = self.conv_block_3(x)
        
        # Before flattening the tensor, to further reduce the parameters
        # we use adaptive average pooling
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        
        x = self.linear_block(x)
        return x


In [19]:

import os
import json


def train(
    model, 
    loss_fn, 
    optimizer, 
    train_dl, 
    val_dl, 
    device, 
    n_epochs, 
    checkpoint_path = None,
    logging_path = None,
    random_seed = 42
):
    fix_random(random_seed)
    if checkpoint_path is not None: os.makedirs(checkpoint_path, exist_ok=True)
    if logging_path is not None: os.makedirs(logging_path, exist_ok=True)

    history = []
    model = model.to(device)
    loss_fn = loss_fn.to(device)


    for epoch in tqdm(range(n_epochs)):
        # Train
        model.train()
        batch_train_accuracies = []
        batch_train_losses = []
        batch_train_times = []
        for i, (images, labels) in enumerate(train_dl):
            if device == "cuda":
                start_time = torch.cuda.Event(enable_timing=True)
                end_time = torch.cuda.Event(enable_timing=True)
            images, labels = images.to(device), labels.to(device)

            if device == "cuda":
                start_time.record()
            optimizer.zero_grad()
            pred_logits = model(images)
            loss = loss_fn(pred_logits, labels)
            loss.backward()
            optimizer.step()
            if device == "cuda":
                end_time.record()
            
            _, preds = torch.max(pred_logits, 1)
            batch_train_accuracies.append((preds == labels).sum().item() / labels.size(0))
            batch_train_losses.append(loss.item())
            if device == "cuda":
                torch.cuda.synchronize()
                batch_train_times.append(start_time.elapsed_time(end_time))


        # Validation
        batch_val_accuracy = 0
        batch_val_loss = 0
        
        model.eval()
        with torch.no_grad():
            for images, labels in val_dl:
                images, labels = images.to(device), labels.to(device)
                preds_logits = model(images)
                _, pred = torch.max(preds_logits, 1)
                batch_val_accuracy += (pred == labels).sum().item()
                batch_val_loss += loss_fn(preds_logits, labels).item() * labels.shape[0]
        batch_val_accuracy = batch_val_accuracy / len(val_dl.dataset)
        batch_val_loss = batch_val_loss / len(val_dl.dataset)

        print(f"Epoch {epoch}, "
                f"train_acc: {np.mean(batch_train_accuracies):.4f}, "
                f"val_acc: {batch_val_accuracy:.4f}, "
                f"train_loss: {np.mean(batch_train_losses):.4f}, "
                f"val_loss: {batch_val_loss:.4f}")
        
        # Checkpoint
        if checkpoint_path is not None:
            torch.save({
                "state_dict": model.state_dict(),
                "architecture": str(model)
            }, os.path.join(checkpoint_path, f"epoch{epoch+1}.ckp.pt"))

        # Logging
        epoch_logs = {
            "epoch": epoch + 1,
            "batch_train_accuracies": batch_train_accuracies,
            "batch_train_losses": batch_train_losses,
            "batch_train_times": batch_train_times,
            "batch_val_accuracy": batch_val_accuracy,
            "batch_val_loss": batch_val_loss
        }
        if logging_path is not None:
            with open(os.path.join(logging_path, f"epoch{epoch+1}.log.json"), "w") as f:
                json.dump(epoch_logs, f, indent=3)
        history.append(epoch_logs)

        # print("Test acc: ", evaluate(model, test_dl, device))
            
    return history

model = FirstModel((INPUT_HEIGHT, INPUT_WIDTH), NUM_CLASSES).to(device)
print(model(train_dset[0][0].unsqueeze(0).to(device)))

logs = train(
    model = model,
    loss_fn = nn.CrossEntropyLoss(),
    optimizer = optim.Adam(model.parameters(), lr=3e-5, weight_decay=1e-5),
    train_dl = train_dl, 
    val_dl = val_dl,
    device = device,
    n_epochs = 100,
    random_seed = 42,
    checkpoint_path = "./checkpoints",
    logging_path = "./logs"
)


tensor([[-0.1004,  0.0555, -0.0185, -0.1042,  0.0835,  0.0452, -0.0722,  0.1008,
         -0.0017, -0.0011, -0.1373,  0.0230,  0.0372, -0.0906,  0.0155, -0.1794,
          0.0249,  0.0167, -0.1061,  0.0326, -0.0333,  0.0961, -0.0480,  0.0239,
          0.0674,  0.0110, -0.0440, -0.0131,  0.0354, -0.0658, -0.0722, -0.0324,
          0.0354,  0.0149,  0.0192,  0.0057, -0.0393, -0.0251,  0.0081,  0.0661,
          0.0285,  0.0266,  0.0835]], grad_fn=<AddmmBackward0>)


  0%|          | 0/100 [06:33<?, ?it/s]


KeyboardInterrupt: 

In [10]:
def print_model_parameters(model):
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param_count = parameter.numel()
        total_params += param_count
    
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param_count = parameter.numel()
        
        print(f"Layer: {name} | Parameters: {param_count/total_params*100:.2f}%")
    print(f"Total Trainable Parameters: {total_params}")

In [11]:
model = FirstModel((INPUT_HEIGHT, INPUT_WIDTH), NUM_CLASSES).to(device)
print_model_parameters(model)

Layer: stem_block.0.weight | Parameters: 0.10%
Layer: stem_block.0.bias | Parameters: 0.00%
Layer: stem_block.2.weight | Parameters: 0.00%
Layer: stem_block.2.bias | Parameters: 0.00%
Layer: stem_block.4.weight | Parameters: 2.24%
Layer: stem_block.4.bias | Parameters: 0.00%
Layer: stem_block.6.weight | Parameters: 0.00%
Layer: stem_block.6.bias | Parameters: 0.00%
Layer: conv_block_1.block.0.weight | Parameters: 3.22%
Layer: conv_block_1.block.0.bias | Parameters: 0.00%
Layer: conv_block_1.block.1.weight | Parameters: 0.00%
Layer: conv_block_1.block.1.bias | Parameters: 0.00%
Layer: conv_block_1.block.3.weight | Parameters: 12.88%
Layer: conv_block_1.block.3.bias | Parameters: 0.01%
Layer: conv_block_1.block.4.weight | Parameters: 0.01%
Layer: conv_block_1.block.4.bias | Parameters: 0.01%
Layer: conv_block_1.block.6.weight | Parameters: 12.88%
Layer: conv_block_1.block.6.bias | Parameters: 0.00%
Layer: conv_block_1.block.7.weight | Parameters: 0.00%
Layer: conv_block_1.block.7.bias | 

In [13]:
from torchsummary import summary

summary(model, (3 , INPUT_HEIGHT, INPUT_WIDTH), device="cpu")


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 114, 114]           9,472
              SiLU-2         [-1, 64, 114, 114]               0
       BatchNorm2d-3         [-1, 64, 114, 114]             128
           Dropout-4         [-1, 64, 114, 114]               0
            Conv2d-5          [-1, 128, 55, 55]         204,928
              SiLU-6          [-1, 128, 55, 55]               0
       BatchNorm2d-7          [-1, 128, 55, 55]             256
           Dropout-8          [-1, 128, 55, 55]               0
            Conv2d-9          [-1, 256, 27, 27]         295,168
           Conv2d-10          [-1, 256, 27, 27]         295,168
      BatchNorm2d-11          [-1, 256, 27, 27]             512
             SiLU-12          [-1, 256, 27, 27]               0
             SiLU-13          [-1, 256, 27, 27]               0
           Conv2d-14          [-1, 512,